In [1]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Ejemplo de datos: frases sobre tecnología
datos = [
    "La inteligencia artificial está transformando el mundo.",
    "Las redes neuronales pueden identificar patrones complejos.",
    "El aprendizaje profundo es una técnica clave en IA.",
    "Python es popular en la ciencia de datos.",
    "Los algoritmos de machine learning mejoran con el tiempo.",
    "La visión por computadora permite a las máquinas 'ver'.",
    "El procesamiento de lenguaje natural facilita la interacción humano-máquina.",
    "Los sistemas de recomendación personalizan la experiencia en línea.",
    "La robótica utiliza IA para realizar tareas complejas.",
    "Los vehículos autónomos emplean IA para navegar.",
    "La seguridad informática se fortalece mediante el uso de IA.",
    "La inteligencia artificial contribuye al desarrollo de la medicina personalizada.",
    "Las ciudades inteligentes utilizan IA para mejorar la vida urbana.",
    "La IA en la agricultura ayuda a optimizar la producción."
]

# Lista de stopwords en español
stop_words = set(stopwords.words('spanish'))

# Tokenización de las frases y eliminación de stopwords
datos_tokenizados = [[palabra for palabra in word_tokenize(frase.lower()) if palabra not in stop_words]
                     for frase in datos]

# Entrenamiento del modelo Word2Vec
modelo = Word2Vec(sentences=datos_tokenizados, vector_size=100, window=5, min_count=1, workers=4)

# Obtener el vector de una palabra
vector_palabra = modelo.wv['python']

# Encontrar palabras similares
palabras_similares = modelo.wv.most_similar('python')

print("Vector de 'python':", vector_palabra)
print("Palabras similares a 'python':", palabras_similares)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicholas.ayala\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicholas.ayala\AppData\Roaming\nltk_data...


Vector de 'python': [ 5.6248726e-03  5.4952400e-03  1.8312103e-03  5.7466258e-03
 -8.9694280e-03  6.5586166e-03  9.2282193e-03 -4.2049107e-03
  1.6067064e-03 -5.2374001e-03  1.0574245e-03  2.7697671e-03
  8.1609664e-03  5.4541335e-04  2.5559543e-03  1.2965827e-03
  8.4043918e-03 -5.7061240e-03 -6.2658801e-03 -3.6310293e-03
 -2.3045607e-03  5.0418666e-03 -8.1245182e-03 -2.8300399e-03
 -8.1986710e-03  5.1500006e-03 -2.5678228e-03 -9.0677571e-03
  4.0718429e-03  9.0164822e-03 -3.0370811e-03 -5.8357748e-03
  3.0194754e-03 -4.3421809e-04 -9.9786399e-03  8.4149186e-03
 -7.3371446e-03 -4.9352939e-03 -2.6536766e-03 -5.4505393e-03
  1.7142271e-03  9.7128889e-03  4.5755468e-03  8.0884127e-03
 -4.6950090e-04  6.4095791e-04 -2.6666471e-03 -8.7800510e-03
  3.4330073e-03  2.0929975e-03 -9.4188582e-03 -4.9675275e-03
 -9.7347703e-03 -5.7200622e-03  4.0658559e-03  8.6448863e-03
  4.1154488e-03  2.3923100e-03  8.1495717e-03 -1.1160529e-03
 -1.3966425e-03 -8.7489663e-03 -1.2374498e-04 -2.5661718e-03
  3.

[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Cargar datos
df = pd.read_excel('excel_movies.xlsx')
df['overview'] = df['overview'].fillna(' ').astype('str')
# Preprocesamiento básico sin dividir en palabras
df['overview_clean'] = df['overview'].str.lower()  # Convertir a minúsculas

# Tokenización por frases
df['overview_frases'] = df['overview_clean'].apply(lambda x: sent_tokenize(x))

# Aplanar la lista de frases para entrenar Word2Vec, que espera una lista de palabras por frase/documento
descripciones_frases = [word_tokenize(frase) for lista_frases in df['overview_frases'] for frase in lista_frases]

# Inicializar el modelo Word2Vec sin entrenar
modelo_word2vec = Word2Vec(vector_size=300, window=10, min_count=1, epochs=15, workers=4)
modelo_word2vec.build_vocab(descripciones_frases)
modelo_word2vec.train(descripciones_frases, total_examples=modelo_word2vec.corpus_count, epochs=modelo_word2vec.epochs)

def obtener_vector_pelicula(frases_pelicula, modelo_param):
    vectores = [modelo_param.wv[word] for frase in frases_pelicula for word in frase if word in modelo_param.wv]
    if not vectores:
        return np.zeros(modelo_param.vector_size)
    vector_pelicula = np.mean(vectores, axis=0)
    return vector_pelicula

# Generar vectores para cada conjunto de frases de películas usando Word2Vec
vectores_peliculas_word2vec = np.array([obtener_vector_pelicula(frases, modelo_word2vec) for frases in df['overview_frases']])

# Función para encontrar películas similares
def peliculas_similares(index, vectores_peliculas_param):
    similitudes = cosine_similarity([vectores_peliculas_param[index]], vectores_peliculas_param)[0]
    indices_similares = np.argsort(similitudes)[::-1][1:6]
    return [(indice, similitudes[indice]) for indice in indices_similares]

# Ejemplo de uso con Word2Vec
indice_pelicula = 867  # Cambiar por el índice de la película de interés
print(f"... buscando similares a '{df.iloc[indice_pelicula]['title']}' con Word2Vec:")
peliculas_sim = peliculas_similares(indice_pelicula, vectores_peliculas_word2vec)
for pelicula_idx in peliculas_sim:
    print(df.iloc[pelicula_idx[0]]['title'])

In [ ]:
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Crear un stemmer en español
stemmer = SnowballStemmer('spanish')
nltk.download('punkt')
nltk.download('stopwords')
# Lista de stopwords en español
stop_words = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    # Tokenizar el texto
    palabras = word_tokenize(texto)
    # Quitar puntuación y stopwords
    palabras_filtradas = [stemmer.stem(palabra.lower()) for palabra in palabras if palabra.isalpha() and palabra.lower() not in stop_words]
    return palabras_filtradas

# Ejemplo de datos: nombres de películas y sus descripciones
peliculas = [
    "El Padrino: Don Vito Corleone es el respetado y temido jefe de una de las cinco familias de la mafia de Nueva York en los años 40. El hombre tiene cuatro hijos: Connie, Sonny, Fredo y Michael, que no quiere saber nada de los negocios sucios de su padre. Cuando otro capo, Sollozzo, intenta asesinar a Corleone, empieza una cruenta lucha entre los distintos clanes.",
    "Toy Story: Los juguetes de Andy, un niño de seis años, temen que un nuevo regalo les sustituya en el corazón de su dueño. Woody, un vaquero que ha sido hasta ahora el juguete favorito, trata de tranquilizarlos hasta que aparece Buzz Lightyear. Lo peor es que el arrogante Buzz se cree que es una auténtico astronauta en plena misión para regresar a su planeta.",
    "Forrest Gump: Sentado en un banco en Savannah, Georgia, Forrest Gump espera al autobús. Mientras éste tarda en llegar, el joven cuenta su vida a las personas que se sientan a esperar con él. Aunque sufre un pequeño retraso mental, esto no le impide hacer cosas maravillosas. Sin entender del todo lo que sucede a su alrededor, Forrest toma partido en los eventos más importantes de la historia de los Estados Unidos.",
    "La lista de Schindler: Oskar Schindler, un empresario alemán, salva la vida de más de mil judíos polacos durante el Holocausto al emplearlos en sus fábricas. La película narra su evolución desde un oportunista indiferente a un héroe improbable y compasivo.",
    "El Señor de los Anillos: La Comunidad del Anillo: En la Tierra Media, Frodo Bolsón, un joven hobbit, es encargado de destruir un anillo poderoso y malvado antes de que caiga en manos del oscuro señor Sauron. La formación de la Comunidad del Anillo tiene como objetivo ayudar a Frodo en su misión.",
    "Matrix: Thomas A. Anderson es un programador de día y un hacker llamado Neo de noche. Sospechando que algo anda mal con el mundo, Neo descubre la verdad sobre la Matrix, una simulación virtual creada para someter a la humanidad, y se une a la lucha contra sus controladores.",
    "Amelie: En París, Amelie Poulain, una joven camarera, decide cambiar la vida de las personas a su alrededor para mejor, mientras lucha con su propia soledad. Sus buenas acciones provocan una cadena de eventos inesperados.",
    "Pulp Fiction: Las vidas de dos sicarios, un boxeador, la esposa de un mafioso y dos bandidos se entrelazan en cuatro historias de violencia y redención.",
    "El club de la lucha: Un empleado de oficina insomne y un carismático vendedor de jabón forman un club de lucha clandestino que se convierte en algo mucho más grande. La película explora temas de consumismo, insatisfacción y la búsqueda de identidad.",
    "Gladiator: En el año 180, el general romano Maximus Decimus Meridius es traicionado cuando el hijo del emperador asesina a su propio padre y se apodera del trono. Reducido a la esclavitud, Maximus se convierte en gladiador y lucha por su venganza.",
    "Interestelar: En un futuro cercano, la Tierra está siendo devastada por desastres naturales. Un equipo de astronautas viaja a través de un agujero de gusano en busca de un nuevo hogar para la humanidad. La película explora temas de amor, sacrificio y la lucha por la supervivencia."
]

peliculas_limpia = [limpiar_texto(pelicula) for pelicula in peliculas]
# Entrenar el modelo Word2Vec
modelo = Word2Vec(peliculas_limpia, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Ejemplo de uso: encontrar palabras similares a 'familia'
palabras_similares = modelo.wv.most_similar(stemmer.stem('familia'), topn=5)
print(palabras_similares)